In [2]:
import os
import numpy as np
import xarray as xr
import pandas as pd
from utulities.fetching_data import extract_weatherbench_input_data
from utulities.inference_model import predict_region_weather
from utulities.subregion import *
import onnx
import onnxruntime as ort
from datetime import datetime, timedelta

#  Downloading the data of the inital field

In [9]:
# Define the input datetime range 
start_date = '2023-01-16T00:00:00' 
end_date = '2023-01-30T00:00:00'
datetime_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Extract input data for each day at 00:00 for the year 2022
extract_weatherbench_input_data(start_date, end_date)

KeyError: "not all values found in index 'time'. Try setting the `method` keyword argument (example: method='nearest')."

# Forecasting for next 24 hr

In [2]:
# Define paths and parameters
input_base_dir = 'input'
#model_path = 'models/pangu_weather_1.onnx'
model_path = 'models/pangu_weather_6.onnx'
start_date = '2022-01-01T00:00:00'
end_date = '2022-01-10T00:00:00'
lat_min=50
lat_max=52
lon_min=2
lon_max=6
region_name = 'BE'

# Run the main function
predict_region_weather(input_base_dir, model_path, start_date, end_date,region_name,lat_min, lat_max, lon_min, lon_max, model_type='6hr')

Running inference for 2022-01-01
2022-01-01 6 hour
2022-01-01 12 hour
2022-01-01 18 hour
2022-01-01 24 hour
CPU Time: 5 minutes 17 seconds
Memory Usage: 1360.546875 MB
Running inference for 2022-01-02
2022-01-02 6 hour
2022-01-02 12 hour
2022-01-02 18 hour
2022-01-02 24 hour
CPU Time: 5 minutes 47 seconds
Memory Usage: 1791.046875 MB
Running inference for 2022-01-03
2022-01-03 6 hour
2022-01-03 12 hour
2022-01-03 18 hour
2022-01-03 24 hour
CPU Time: 5 minutes 22 seconds
Memory Usage: 3059.0 MB
Running inference for 2022-01-04
2022-01-04 6 hour
2022-01-04 12 hour
2022-01-04 18 hour
2022-01-04 24 hour
CPU Time: 5 minutes 35 seconds
Memory Usage: 1524.65625 MB
Running inference for 2022-01-05
2022-01-05 6 hour
2022-01-05 12 hour
2022-01-05 18 hour
2022-01-05 24 hour
CPU Time: 5 minutes 42 seconds
Memory Usage: 1750.5625 MB
Running inference for 2022-01-06
2022-01-06 6 hour
2022-01-06 12 hour
2022-01-06 18 hour
2022-01-06 24 hour
CPU Time: 5 minutes 22 seconds
Memory Usage: 3222.828125 MB


# coordinates checking 

In [15]:
lat_min = 50
lat_max = 51.5
lon_min = 2.5
lon_max = 6.5
region_name = 'BE'

map_bounding_box('/Users/adacanaydin/Documents/Python/pangu-weather-operational/', region_name, lat_min, lat_max, lon_min, lon_max)

Bounding Box for the Region:
Latitude Range: [50, 51.5]
Longitude Range: [2.5, 6.5]


In [ ]:
file_path = "/Users/adacanaydin/Documents/Python/pangu-weather-operational/input/2022-01-01 00:00:00/upper.nc"
ds = xr.open_dataset(file_path)
upper_variables = ['geopotential', 'specific_humidity', 'temperature', 'u_component_of_wind', 'v_component_of_wind']
upper_data = np.zeros((5,13, 721, 1440), dtype=np.float32)
for i, var in enumerate(upper_variables):
                upper_data[i] = ds[var.lower()].values.astype(np.float32)

In [8]:
lat_values = np.linspace(90, -90, 721)  # Latitude values from 90 to -90 with 721 values
lon_values = np.linspace(0.0000e+00, 3.5975e+02, 1440)  # Longitude values from 0 to 360 with 1440 values

lat_indices = np.where((lat_values >= lat_min) & (lat_values <= lat_max))[0]
lon_indices = np.where((lon_values >= lon_min) & (lon_values <= lon_max))[0]

latitude_list = lat_values[lat_indices]
longitude_list = lon_values[lon_indices]

In [80]:
# Selecting latitude and longitude indices from latitude_list and longitude_list
lat_indices_ds = ds.sel(latitude=latitude_list).latitude.values
lon_indices_ds = ds.sel(longitude=longitude_list).longitude.values

# Selecting geopotential variable values
geopotential_values = ds.geopotential.sel(latitude=lat_indices_ds, longitude=lon_indices_ds,level=1000)
geopotential_values

<xarray.DataArray 'geopotential' (latitude: 9, longitude: 17)>
array([[1683.7313, 1700.709 , 1718.0452, 1734.7837, 1748.7722, 1758.9349,
        1765.8694, 1770.891 , 1773.9995, 1776.1516, 1779.2601, 1784.6404,
        1792.4119, 1801.4984, 1809.6285, 1814.1719, 1815.2479],
       [1725.5775, 1741.3595, 1757.7393, 1775.1951, 1792.2922, 1806.8787,
        1816.9218, 1821.7042, 1822.1824, 1821.1063, 1820.8672, 1822.6606,
        1826.4866, 1832.9429, 1840.9534, 1848.0074, 1853.1486],
       [1764.3151, 1780.097 , 1795.9987, 1813.5741, 1831.7472, 1847.888 ,
        1859.0071, 1864.5068, 1865.1046, 1863.4308, 1861.1592, 1859.9635,
        1861.0396, 1865.4633, 1872.3978, 1880.2888, 1887.4624],
       [1801.0201, 1817.6392, 1835.3341, 1853.3877, 1870.1262, 1883.9952,
        1894.0383, 1899.4185, 1900.2554, 1898.2229, 1895.9513, 1895.3534,
        1897.6251, 1902.288 , 1908.7443, 1916.2765, 1923.8088],
       [1837.7252, 1856.0181, 1875.626 , 1894.5165, 1910.2985, 1921.6567,
        1929.0695, 1932.4172, 1932.5367, 1931.5803, 1932.0585, 1935.0475,
        1939.3517, 1944.1342, 1950.3513, 1958.4814, 1965.4159],
       [1878.1367, 1894.2773, 1911.9723, 1930.5043, 1947.2427, 1959.1987,
        1965.4159, 1967.6875, 1968.644 , 1970.1984, 1973.1874, 1978.0894,
        1983.9478, 1990.0454, 1998.1755, 2007.3816, 2012.6423],
       [1919.9829, 1932.8954, 1946.1666, 1962.1877, 1978.9263, 1992.317 ,
        1999.7297, 2003.7948, 2007.2621, 2011.4467, 2016.5878, 2023.5223,
        2032.0111, 2040.8585, 2050.4236, 2059.3906, 2063.097 ],
       [1955.6119, 1968.7635, 1980.8392, 1993.1539, 2006.7838, 2020.1746,
        2030.4568, 2037.2717, 2043.0106, 2049.706 , 2057.8362, 2067.7598,
        2078.5203, 2088.085 , 2095.6174, 2100.5193, 2100.041 ],
       [1983.1108, 1998.8928, 2012.7618, 2024.0005, 2035.2393, 2047.6736,
        2058.9124, 2067.4011, 2075.292 , 2084.6177, 2095.0195, 2105.8997,
        2116.66  , 2126.2249, 2131.9639, 2131.2463, 2124.9097]], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 2.0 2.25 2.5 2.75 3.0 ... 5.25 5.5 5.75 6.0
  * latitude   (latitude) float32 52.0 51.75 51.5 51.25 ... 50.5 50.25 50.0
    level      int32 1000
    time       datetime64[ns] ...
Attributes:
    long_name:      Geopotential
    short_name:     z
    standard_name:  geopotential
    units:          m**2 s**-2